# Practical 6.1

# Character-level Sequence-to-Sequence (Seq2Seq) Model

# Application-1: Machine Translation  

We will create a translation model with Recurrent Neural Networks (RNN) by using character sequences as the input of model. 
For building this model, we will need 2 sub models:
* Encoder model: encode text input from source language (English) to a "Neural codes" 
* Decoder model: predict text in target language (Dutch), given the encoding state from encoder model 

![Image](rnn_char_translation.png?raw=true)

## References:

[1] Sutskever, Ilya, Oriol Vinyals, and Quoc V. Le. "Sequence to sequence learning with neural networks." Advances in neural information processing systems. 2014.

[2] Cho, Kyunghyun, et al. "Learning phrase representations using RNN encoder-decoder for statistical machine translation." arXiv preprint arXiv:1406.1078 (2014).

## 1. Data preprocessing

In [3]:
from __future__ import print_function

import os
import sys
import numpy as np

from keras.models import Model
from keras.layers import Input, LSTM, Dense, Embedding, GRU, Lambda, Bidirectional, concatenate

Using TensorFlow backend.


Data set is originally from http://www.manythings.org/anki/.

We will use bilingual sentence pairs of English - Dutch, provided here: https://storage.googleapis.com/trl_data/nld.txt

In [0]:
file_to_read = 'nld.txt'

Example of original format of text for training the model:

```
Hi.	Hoi.
Run!	Ren!
Who?	Wie?
Wow!	Da's niet gek!
Fire!	Vuur!
Fire!	Brand!
Help!	Help!
Jump.	Spring.
Stop!	Stop!
Stop!	Halt!
Hello!	Hallo.
Hello!	Goedemorgen!
```


We will feed two (2) type of inputs for this model.

* `input_texts` : input sequences for encoder model (English text)
* `target_texts` : input sequences for decoder model (Dutch text)

We also need two (2) look up dictionaries:

* `input_characters` : dictionary index for encoder model
* `target_characters` : dictionary index for decoder model

In [0]:
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()

In [0]:
#reading text line by line
lines = open(file_to_read).read().split('\n')

We only use 10000 samples (lines) from original text to train the model. 

* Notice that for `target_texts`, we add character `'\t'` at the beginning of text and character `'\n'` at the end of text sequences. 

Why do we need to add these two characters? 

The prediction layer in our RNN model need to predict one time step ahead when learn character in current time step. 


In [0]:
num_samples = 10000  # Number of samples to train on.

for line in lines[: min(num_samples, len(lines) - 1)]:
    
    english_text, dutch_text = line.split('\t')
    
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = '\t' + dutch_text + '\n'
    
    input_texts.append(english_text)
    target_texts.append(target_text)
    
    for char in english_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

In [0]:
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))

In [12]:
# number of characters in input sequences
len(input_characters)

71

In [13]:
# number of characters in target sequences
len(target_characters)

78

Notice that in this example we treat `uppercase` and `lowercase` characters as different characters even if they point to the same character. 

For instance, character `'W'` and `'w'` will be treated as 2 different characters in our input sequences.

In [0]:
#text in source language (english) as input of model
input_texts[:5]

In [0]:
#text in target language (dutch) as input of model
target_texts[:5]

### Look up index for encoder and decoder model

* `enc_index_char` stores character index for input sequences (encoder model)
* `dec_index_char` stores character index for target sequences (decoder model)

You may also use one look-up index for both sub-models.

In [0]:
#dictionary index (character-level) for model input
enc_index_char = dict([(i, char) for i, char in enumerate(input_characters)])

In [0]:
#dictionary index (character-level) for target output
dec_index_char = dict([(i, char) for i, char in enumerate(target_characters)])

We also need to create reversed version of character-index dictionary to map text sequences into integer format.

In [0]:
enc_char_index = dict((char,i) for i, char in enc_index_char.items())
dec_char_index = dict((char,i) for i, char in dec_index_char.items())

In [0]:
list(reverse_input_char_index.items())[:5]

In [0]:
list(reverse_target_char_index.items())[:5]

### Preparing training sequences (one-hot vector format)

In [0]:
#number of unique character as model input
num_encoder_tokens = len(input_characters)

#number of unique character as target output
num_decoder_tokens = len(target_characters)

#maximum sequence length for encoder
#based on maximum length of input text 
max_encoder_seq_length = max([len(txt) for txt in input_texts])

#maximum sequence length for decoder
#based on maximum length of output text 
max_decoder_seq_length = max([len(txt) for txt in target_texts])

* Notice that input and output sequences for our encoder and decoder models are matrices with 3D dimension size (number of samples, sequence length, vocabulary size). 

We will feed sequences as one-hot vector encoding, which represents the occurrence of character in corresponding sequence. The matrix will have value `1` at `i-th` `char_index` of current character.

* Notice that this one-hot encoding step is interchangeble with Lambda layer in Practical 5.2 and 5.3

In [0]:
encoder_input_data = np.zeros((len(input_texts), max_encoder_seq_length, num_encoder_tokens),
                              dtype='float32')

In [0]:
decoder_input_data = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens),
                              dtype='float32')


In [0]:
decoder_target_data = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens),
                               dtype='float32')

In [0]:
text_pair = zip(input_texts, target_texts)
for i, (input_text, target_text) in enumerate(text_pair):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, reverse_input_char_index[char]] = 1.
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, reverse_target_char_index[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
             decoder_target_data[i, t - 1, reverse_target_char_index[char]] = 1.

## 2. Translation model

In [0]:
batch_size = 64  # Batch size for training.
epochs = 200  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.

## 2.a. Encoder model

We will use hidden state `h` and memory state `c` from recurrent layer of encoderas additional input of decoder model.

In [0]:
# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens), name='encoder-input')
encoder = LSTM(latent_dim, name='lstm-encoder', return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

## 2.b. Decoder model

* Set up the decoder, using `encoder_states` as initial state.
* We won't use decoder states in training stage, but we will use them in inference stage later.

In [0]:
decoder_inputs = Input(shape=(None, num_decoder_tokens), name='decoder-input')

decoder_lstm = LSTM(latent_dim, name='lstm-decoder', return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax', name='prediction-layer')
decoder_outputs = decoder_dense(decoder_outputs)

## Encoder - Decoder model

In [0]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

### Compile model

In [0]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

In [28]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
encoder-input (InputLayer)      (None, None, 71)     0                                            
__________________________________________________________________________________________________
decoder-input (InputLayer)      (None, None, 78)     0                                            
__________________________________________________________________________________________________
lstm-encoder (LSTM)             [(None, 256), (None, 335872      encoder-input[0][0]              
__________________________________________________________________________________________________
lstm-decoder (LSTM)             [(None, None, 256),  343040      decoder-input[0][0]              
                                                                 lstm-encoder[0][1]               
          

### Train model

In [29]:
# model training
model.fit([encoder_input_data, decoder_input_data], decoder_target_data, 
          batch_size=batch_size, epochs=epochs, validation_split=0.2)


Train on 8000 samples, validate on 2000 samples
Epoch 1/200
8000/8000 [==============================] - 18s 2ms/step - loss: 1.1128 - val_loss: 1.2334
Epoch 2/200
8000/8000 [==============================] - 17s 2ms/step - loss: 0.8864 - val_loss: 1.0471
Epoch 3/200
8000/8000 [==============================] - 17s 2ms/step - loss: 0.7728 - val_loss: 0.9665
Epoch 4/200
1856/8000 [=====>........................] - ETA: 11s - loss: 0.73758000/8000 [==============================] - 17s 2ms/step - loss: 0.7186 - val_loss: 0.9256
Epoch 5/200
8000/8000 [==============================] - 17s 2ms/step - loss: 0.6853 - val_loss: 0.8757
Epoch 6/200
8000/8000 [==============================] - 17s 2ms/step - loss: 0.6510 - val_loss: 0.8403
Epoch 7/200
6336/8000 [======================>.......] - ETA: 3s - loss: 0.62828000/8000 [==============================] - 17s 2ms/step - loss: 0.6233 - val_loss: 0.8221
Epoch 8/200
8000/8000 [==============================] - 17s 2ms/step - loss: 0.5997 - va

7104/8000 [=========================>....] - ETA: 1s - loss: 0.15858000/8000 [==============================] - 17s 2ms/step - loss: 0.1592 - val_loss: 0.9136
Epoch 68/200
8000/8000 [==============================] - 17s 2ms/step - loss: 0.1571 - val_loss: 0.9309
Epoch 69/200
8000/8000 [==============================] - 17s 2ms/step - loss: 0.1547 - val_loss: 0.9351
Epoch 70/200
7104/8000 [=========================>....] - ETA: 1s - loss: 0.15108000/8000 [==============================] - 17s 2ms/step - loss: 0.1520 - val_loss: 0.9360
Epoch 71/200
8000/8000 [==============================] - 17s 2ms/step - loss: 0.1493 - val_loss: 0.9456
Epoch 72/200
8000/8000 [==============================] - 17s 2ms/step - loss: 0.1465 - val_loss: 0.9538
Epoch 73/200
7104/8000 [=========================>....] - ETA: 1s - loss: 0.14618000/8000 [==============================] - 17s 2ms/step - loss: 0.1482 - val_loss: 0.9573
Epoch 74/200
8000/8000 [==============================] - 17s 2ms/step - loss

6976/8000 [=========================>....] - ETA: 1s - loss: 0.05788000/8000 [==============================] - 17s 2ms/step - loss: 0.0583 - val_loss: 1.3064
Epoch 134/200
8000/8000 [==============================] - 17s 2ms/step - loss: 0.0577 - val_loss: 1.3110
Epoch 135/200
8000/8000 [==============================] - 17s 2ms/step - loss: 0.0572 - val_loss: 1.3103
Epoch 136/200
6976/8000 [=========================>....] - ETA: 1s - loss: 0.05498000/8000 [==============================] - 17s 2ms/step - loss: 0.0557 - val_loss: 1.3168
Epoch 137/200
8000/8000 [==============================] - 17s 2ms/step - loss: 0.0560 - val_loss: 1.3193
Epoch 138/200
8000/8000 [==============================] - 17s 2ms/step - loss: 0.0545 - val_loss: 1.3272
Epoch 139/200
6976/8000 [=========================>....] - ETA: 1s - loss: 0.05398000/8000 [==============================] - 17s 2ms/step - loss: 0.0545 - val_loss: 1.3355
Epoch 140/200
8000/8000 [==============================] - 17s 2ms/step

6976/8000 [=========================>....] - ETA: 2s - loss: 0.02948000/8000 [==============================] - 17s 2ms/step - loss: 0.0297 - val_loss: 1.5245
Epoch 200/200
8000/8000 [==============================] - 17s 2ms/step - loss: 0.0292 - val_loss: 1.5160


In [ ]:
# Save model
model.save('char_translation_model.h5')

In [0]:
model.save_weights('weights_char_translation_model.hdf5')

## 3. Inference mode: Teacher forcing

### 3.1. Re-define encoder model

In [0]:
encoder_model = Model(encoder_inputs, encoder_states)

In [0]:
encoder_model.save('encoder_char_translation.h5')

### 3.2. Re-define decoder model to do the inference

In [0]:
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=
                                                 decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model([decoder_inputs] + decoder_states_inputs, 
                      [decoder_outputs] + decoder_states)

In [ ]:
decoder_model.save(os.path.join('decoder_char_translation.h5')

### 3.3. Translate sentence (English to Dutch)

Notice that we are updating the inference by feeding one-by-one character into decoder model as `teacher forcing` method.

In [0]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, reverse_target_char_index['\t']] = 1.
    
    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = target_token_index[sampled_token_index]
        decoded_sentence += sampled_char
        
        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
            len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True
        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.
        
        # Update states
        states_value = [h, c]
    return decoded_sentence

Example of encoding-decoding from 10 first samples of training sets

In [71]:
for seq_index in range(100,150):
    # Take one sequence (part of the training test)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)

-
Input sentence: I'm home.
Decoded sentence: Ik ben thuis.

-
Input sentence: I'm lost.
Decoded sentence: Ik ben de weg kwijt.

-
Input sentence: I'm sick.
Decoded sentence: Ik ben ziek.

-
Input sentence: I'm tall.
Decoded sentence: Ik ben lang.

-
Input sentence: I'm weak.
Decoded sentence: Ik ben zwak.

-
Input sentence: It hurts.
Decoded sentence: Het doet pijn.

-
Input sentence: It's his.
Decoded sentence: Het is de zijne.

-
Input sentence: It's his.
Decoded sentence: Het is de zijne.

-
Input sentence: It's hot.
Decoded sentence: Het is niet kamorie.

-
Input sentence: It's new.
Decoded sentence: Het is nieuw.

-
Input sentence: Keep out!
Decoded sentence: Verboden toegang.

-
Input sentence: Keep out!
Decoded sentence: Verboden toegang.

-
Input sentence: Let's go!
Decoded sentence: Laten we gaan!

-
Input sentence: Let's go!
Decoded sentence: Laten we gaan!

-
Input sentence: Let's go.
Decoded sentence: Laten we gaan.

-
Input sentence: Let's go.
Decoded sentence: Laten we g